## 자연어 전처리-영어

관련 패키지 로드

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer#토큰화 클래스
from tensorflow.keras.preprocessing.sequence import pad_sequences#시퀀스 패딩화
from sklearn.preprocessing import LabelEncoder#라벨인코더 = 카테고리 데이터를 수치화함
from sklearn.model_selection import train_test_split#학습/테스트 
from tensorflow.keras.utils import to_categorical#원핫인코딩화 클래스
import numpy as np#넘파이
from nltk.corpus import stopwords#불용어
from bs4 import BeautifulSoup#웹
import re#정규화 클래스

전처리 메소드(함수) 구축

In [2]:
def preprocessing(X_text, remove_stopwords=False):
    X_text = BeautifulSoup(X_text, 'lxml').get_text()#불용어 처리
    X_text = re.sub("[^a-zA-Z]", " ", X_text)#불용어처리
    words = X_text.lower().split()#단어모음에 소문자화한 토큰 추가
    if remove_stopwords:
        stops = set(stopwords.words('english'))#불용어처리
        #stops.add(불용어 문자열)
        words = [w for w in words if not w in stops]#리스트(words) 새로재정의
        clean_text = ' '.join(words)#새변수(clean)에 리스트(words) 추가
    else:
        clean_text = ' '.join(words)
        
    return clean_text#변수 반환

데이터 로드

In [5]:
import pandas as pd
data = pd.read_csv('spam (1).csv',encoding='latin1')[['v1','v2']]
data=data.rename(columns = {'v1':'y', 'v2' : 'X'})#컬럼명을 재정의

In [6]:
data['clean_X']=data['X'].apply(lambda x : preprocessing(X_text=x, remove_stopwords=True))
#데이터전체를 전처리하고, 카테고리화

In [23]:
data['clean_X']

0       go jurong point crazy available bugis n great ...
1                                 ok lar joking wif u oni
2       free entry wkly comp win fa cup final tkts st ...
3                     u dun say early hor u c already say
4                  nah think goes usf lives around though
                              ...                        
5567    nd time tried contact u u pound prize claim ea...
5568                            b going esplanade fr home
5569                                pity mood suggestions
5570    guy bitching acted like interested buying some...
5571                                       rofl true name
Name: clean_X, Length: 5564, dtype: object

In [7]:
data['y_name']=data['y']
data['encoder_y']=LabelEncoder().fit_transform(data['y'])#카테고리 데이터를 수치형 데이터로 변환
data['categorical_y']=list(to_categorical(data['encoder_y']))#원핫인코딩화를 한 리스트를 다시 카테고리화

In [22]:
data['categorical_y']

0       [1.0, 0.0]
1       [1.0, 0.0]
2       [0.0, 1.0]
3       [1.0, 0.0]
4       [1.0, 0.0]
           ...    
5567    [0.0, 1.0]
5568    [1.0, 0.0]
5569    [1.0, 0.0]
5570    [1.0, 0.0]
5571    [1.0, 0.0]
Name: categorical_y, Length: 5564, dtype: object

In [8]:
data.isnull().values.any()#결측치 찾기

False

In [9]:
data['clean_X'].nunique(), data['y'].nunique()#X데이터,Y데이터의 고유값들의 종류의수를 나타냄

(5055, 2)

결측치 처리

In [10]:
data['clean_X'] = data['clean_X'].str.replace("[^a-zA-Z0-9 ]","")#공백화
data['clean_X'] = data['clean_X'].str.replace('^ +', "")#공백화
data['clean_X'].replace('', np.nan, inplace=True)#공백=nan화
data = data.dropna(how = 'any')#결측치 제거화

C:\Users\ho316\AppData\Local\Temp/ipykernel_6556/835016061.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['clean_X'] = data['clean_X'].str.replace("[^a-zA-Z0-9 ]","")
C:\Users\ho316\AppData\Local\Temp/ipykernel_6556/835016061.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data['clean_X'] = data['clean_X'].str.replace('^ +', "")


In [11]:
Y=data['encoder_y']#카테고리 데이터를 재정의
#Y=to_categorical(data['encoder_y'])

In [12]:
X=data['clean_X']#재정의

In [25]:
type(X)#아직 카테고리 데이터

pandas.core.series.Series

학습, 테스트 데이터

In [13]:
#학습데이터 테스트 데이터 정의
x_data,test_x,y_data,test_y = train_test_split(X,Y,test_size=0.3,random_state=0)

In [14]:
#학습데이터 검증데이터정의
train_x,val_x,train_y,val_y = train_test_split(x_data,y_data,test_size=0.2,random_state=0)

In [15]:
token=Tokenizer()#토큰화 클래스
token.fit_on_texts(train_x)#토큰학습

In [16]:
token_train_x=token.texts_to_sequences(train_x)#학습데이터 정수화
token_test_x=token.texts_to_sequences(test_x)#테스트데이터 정수화2
token_val_x=token.texts_to_sequences(val_x)#검증데이터 정수화3

In [17]:
train_inputs = pad_sequences(token_train_x)#패딩화
test_inputs = pad_sequences(token_test_x)#패딩화
val_inputs = pad_sequences(token_val_x)#패딩화